In [2]:
import pandas as pd

In [3]:
files = [
    "data/swin_base_patch4_window7_224_imagenet_fp32_WindowAttention_1.00e-03.csv", 
    "data/swin_base_patch4_window7_224_imagenet_fp32_SwinTransformerBlock_1.00e-03.csv",
    "data/swin_base_patch4_window7_224_imagenet_fp32_Mlp_1.00e-03.csv",
    "data/vit_base_patch16_224_imagenet_fp32_Attention_1.00e-03.csv", 
    "data/vit_base_patch16_224_imagenet_fp32_Block_1.00e-03.csv",
    "data/vit_base_patch16_224_imagenet_fp32_Mlp_1.00e-03.csv",
]

In [4]:
for file in files:
    print("="*50)
    if "SwinTransformerBlock" in file:
        print(f" [INFO] SwinTransformerBlock")
    elif "Mlp" in file:
        print(f" [INFO] Mlp")
    elif "WindowAttention" in file:
        print(f" [INFO] WindowAttention")
    else:
        print(f" [INFO] Unknown")
    
    df = pd.read_csv(file)

    accuracy_wo_fault = (df["prediction_without_fault"] == df["ground_truth"]).sum() / len(df)
    accuracy_with_fault = (df["prediction_with_fault"] == df["ground_truth"]).sum() / len(df)

    print(f" [INFO] Accuracy without fault: {accuracy_wo_fault * 100:.2f}%")
    print(f" [INFO] Accuracy with fault: {accuracy_with_fault * 100:.2f}%")
    print()
    count = 0
    for i, row in df.iterrows():
        out_wo_fault, out_with_fault = int(row["prediction_without_fault"]), int(row["prediction_with_fault"])
        if out_with_fault != out_wo_fault:
            count += 1
            print(f" [INFO] Critical error for image {i}. Expected {out_wo_fault} but got {out_with_fault}. Ground truth: {int(row['ground_truth'])}")

    print(f" [INFO] Total critical errors: {count}")

 [INFO] WindowAttention
 [INFO] Accuracy without fault: 85.27%
 [INFO] Accuracy with fault: 85.28%

 [INFO] Critical error for image 3066. Expected 45 but got 29. Ground truth: 61
 [INFO] Critical error for image 3417. Expected 68 but got 66. Ground truth: 68
 [INFO] Critical error for image 8671. Expected 173 but got 253. Ground truth: 173
 [INFO] Critical error for image 9335. Expected 184 but got 265. Ground truth: 186
 [INFO] Critical error for image 11116. Expected 222 but got 258. Ground truth: 222
 [INFO] Critical error for image 19670. Expected 108 but got 393. Ground truth: 393
 [INFO] Critical error for image 21793. Expected 876 but got 502. Ground truth: 435
 [INFO] Critical error for image 26728. Expected 729 but got 651. Ground truth: 534
 [INFO] Critical error for image 26733. Expected 567 but got 896. Ground truth: 534
 [INFO] Critical error for image 30140. Expected 702 but got 602. Ground truth: 602
 [INFO] Critical error for image 33937. Expected 678 but got 837. Grou

In [ ]:
dataset = "imagenet"
precision = "fp32"
models = [
    "swin_base_patch4_window7_224",
    "vit_base_patch16_224",
]

for model in models:
    file = f"data/{model}_{dataset}_{precision}_correct_predictions.csv"
    if model == "swin_base_patch4_window7_224":
        to_open = "data/swin_base_patch4_window7_224_imagenet_fp32_WindowAttention_1.00e-03.csv"
    elif model == "vit_base_patch16_224":
        top_open = "data/vit_base_patch16_224_imagenet_fp32_Attention_1.00e-03.csv"

    df = pd.read_csv(to_open)
    df = df[df["prediction_without_fault"] == df["ground_truth"]]
    df.to_csv(file, index=True)

In [18]:
import torch
import os

INDICES_IDX, VALUES_IDX = 0, 1

dataset = "imagenet"
precision = "fp32"
models = [
    "swin_base_patch4_window7_224",
    "vit_base_patch16_224",
]

final_dfs = []
for model in models:
    dfs = []
    logits = [f for f in os.listdir(f"data/top5prob") if f"{model}_{dataset}_{precision}" in f]
    
    # logit = next(iter(logits))
    # logit = torch.load(f"data/top5prob/{logit}")
    # print(logit.values[0])
    for logit_path in logits:
        logit = torch.load(f"data/top5prob/{logit_path}")
        dfs.append({
            "model": model,
            "dataset": dataset,
            "precision": precision,
            "top1_prob": logit[VALUES_IDX][0].item(),
            "top2_prob": logit[VALUES_IDX][1].item(),
            "top_diff": logit[VALUES_IDX][0].item() - logit[VALUES_IDX][1].item(),
            "top1_class": int(logit[INDICES_IDX][0].item()),
            "top2_class": int(logit[INDICES_IDX][1].item()),
        })

    df = pd.DataFrame(dfs)
    final_dfs.append((model, df))
    df.to_csv(f"data/{model}_{dataset}_{precision}_top5_prob.csv", index=False)



In [30]:
for model, df in final_dfs:
    print("="*50)
    print(f" [INFO] {model}")
    print(f" Min diff: {df['top_diff'].min():.2f}")
    print(f" Max diff: {df['top_diff'].max():.2f}")
    print(f" Mean diff: {df['top_diff'].mean():.2f}")
    print(f" Median diff: {df['top_diff'].median():.2f}")
    print(f" Q25% diff: {df['top_diff'].quantile(0.25):.2f}")
    print(f" Q75% diff: {df['top_diff'].quantile(0.75):.2f}")
    print(f" Q5% diff: {df['top_diff'].quantile(0.05):.2f}")
    print(f" Q10% diff: {df['top_diff'].quantile(0.10):.2f}")


 [INFO] swin_base_patch4_window7_224
 Min diff: 0.00
 Max diff: 1.00
 Mean diff: 0.77
 Median diff: 0.86
 Q25% diff: 0.73
 Q75% diff: 0.90
 Q5% diff: 0.23
 Q10% diff: 0.43
 [INFO] vit_base_patch16_224
 Min diff: 0.00
 Max diff: 0.99
 Mean diff: 0.74
 Median diff: 0.84
 Q25% diff: 0.71
 Q75% diff: 0.88
 Q5% diff: 0.20
 Q10% diff: 0.39
